# Names 
Spencer Bernardo-Cheng
Max Goldstein
Robbie Keehan

# Data Preparation

In [1]:
import numpy as np
import pandas as pd
datafb = pd.read_csv("./FB.csv")
datagoogle = pd.read_csv("./GOOGL.csv")
datamsft = pd.read_csv("./MSFT.csv")
dataam = pd.read_csv("./AMZN (1).csv")
datanflx = pd.read_csv("./NFLX.csv")

del datafb['Date']
del datagoogle['Date']
del datamsft['Date']
del datanflx['Date']


data = pd.concat([datafb, datagoogle, datamsft,datanflx, dataam ], axis=1, sort=False)
data = data[89:]
del data['Change']
del data['Gain']
del data['Loss']
del data['Average Gain']
del data['Average Loss']
del data['RS']
del data['DM+1']
del data['DM-1']
del data['TR-14']
del data['DM+1-14']
del data['DM-1+14']
del data['dl+1-14']
del data['dl-1-14']
del data['dl14diff']
del data['dl14sum']
del data['DX']
del data['TR']
ogdates = []
for val in data['Date']:
    ogdates.append(val)
del data['Date']

predictedDays = 14
up = .03
down =-.03
## Time difference the data
colheaders = data.columns[0:-3]
og = []
for val in data['Close-amzn']:
    og.append(val)
for header in colheaders: 
    for i in range(1, len(data)):
        perchange =((data[header].iloc[i]) - (data[header].iloc[i - 1]))/float(data[header].iloc[i-1])
        data.set_value(89+i,"pc-"+header,perchange)
#         data["pc"+header].at[90+i]= perchange
data = data[1:]
ydata = data['pc-Close-amzn']
# ydata = ydata.apply(lambda x:2 if x>=up else (1 if (x< up and x > down) else 0))
ydata = pd.DataFrame(ydata)
ydata['newy'] = ""
# created target data in format to match output of multiple timesteps of 14 days ahead
# [0,1,2,3,4,5,6,7]
# [1,2,3,4,5,6,7,8]
for index, row in ydata.iterrows():
    temp = np.array(ydata['pc-Close-amzn'][index-90:index-90+predictedDays].values)
    ydata['newy'].at[index] = temp
    
# removed last 14 because their targets are not able to created
ydata = ydata[0:-(predictedDays-1)]
data = data[0:-(predictedDays-1)]

del ydata['pc-Close-amzn']

xdata =data[['RSI-14 Day', 'ADX',
       'pc-Open-fb', 'pc-High-fb', 'pc-Low-fb', 'pc-Close-fb',
       'pc-Adj Close-fb', 'pc-Volume-fb', 'pc-Open-googl', 'pc-High-googl',
       'pc-Low-googl', 'pc-Close-googl', 'pc-Adj Close-googl',
       'pc-Volume-googl', 'pc-Open-msft', 'pc-High-msft', 'pc-Low-msft',
       'pc-Close-msft', 'pc-Adj Close-msft', 'pc-Volume-msft', 'pc-Open-nflx',
       'pc-High-nflx', 'pc-Low-nflx', 'pc-Close-nflx', 'pc-Adj Close-nflx',
       'pc-Volume-nflx', 'pc-Open-amzn', 'pc-High-amzn', 'pc-Low-amzn',
       'pc-Close-amzn', 'pc-Adj Close-amzn', 'pc-Volume-amzn', 'pc-SMA-7',
       'pc-SMA-21', 'pc-SMA-90']]


data = data[0:-1]
xdata = xdata[0:-1]
ydata = ydata[0:-1]
# data = data.T


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


# One Hot Encoding The Target Data

In [2]:
from keras.utils.np_utils import to_categorical
print(ydata.head())
# ydata['newy'] = ydata['newy'].apply(lambda x: to_categorical(x,3))
print(ydata)

Using TensorFlow backend.


                                                 newy
90  [-0.03126685348001901, -0.023788658883803833, ...
91  [-0.023788658883803833, -0.0031430016910281716...
92  [-0.0031430016910281716, -0.007616083827630422...
93  [-0.007616083827630422, 0.00750945697512207, -...
94  [0.00750945697512207, -0.0009828446509949397, ...
                                                   newy
90    [-0.03126685348001901, -0.023788658883803833, ...
91    [-0.023788658883803833, -0.0031430016910281716...
92    [-0.0031430016910281716, -0.007616083827630422...
93    [-0.007616083827630422, 0.00750945697512207, -...
94    [0.00750945697512207, -0.0009828446509949397, ...
95    [-0.0009828446509949397, 0.014552771053924872,...
96    [0.014552771053924872, -0.010667093404438537, ...
97    [-0.010667093404438537, -0.01980806976169732, ...
98    [-0.01980806976169732, -0.0259166708333332, 0....
99    [-0.0259166708333332, 0.0022670844480582584, -...
100   [0.0022670844480582584, -0.02483885898885134, ...
101 

# Min-Max Scaling the Time Series

In [3]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(xdata.iloc[:,:].values)
print(x_scaled)



[[0.35811382 0.15022488 0.40722014 ... 0.53721892 0.45230866 0.64160589]
 [0.26414926 0.12331708 0.35961381 ... 0.50635447 0.41867806 0.57327146]
 [0.25300192 0.10448737 0.40286589 ... 0.51153038 0.41312322 0.576277  ]
 ...
 [0.3029723  0.1344608  0.461637   ... 0.50352112 0.48231708 0.23750753]
 [0.31038595 0.1153046  0.38866776 ... 0.50862139 0.47320208 0.21748327]
 [0.25243424 0.13533161 0.42314792 ... 0.49730187 0.44639693 0.21806241]]


# Utilizing Time Series Generator

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM, Lambda, Dropout,GRU
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 2
series = np.array(x_scaled)
target = np.array(ydata)
generator = TimeseriesGenerator(series, target, length=90, batch_size=1)
xtime = []
ytarget =[]
for i in range(len(generator)):
    x, y = generator[i]
    xtime.append(x[0])
xtime = np.array(xtime)

target = target[90:]
newtarget = []
for val in target:
    newtarget.append(np.array(val[0]))
ytarget = np.array(newtarget)
print(ytarget.shape)


(1698, 14)


# Time Series Fold Splits

In [7]:
from sklearn.model_selection import train_test_split
import keras.utils
from keras import utils as np_utils
from sklearn.model_selection import TimeSeriesSplit



# LSTM

In [8]:
from sklearn.model_selection import KFold

model = Sequential()
model.add(LSTM(100,dropout=.2,activation='relu', input_shape=(90, 35), return_sequences=True))
model.add(Lambda(lambda x: x[:, -predictedDays:, :]))
model.add(Dense(1))

model.summary()
model.compile(optimizer='adam', loss='mse')

kfold = KFold(n_splits=10, shuffle=False, random_state=None)
# fit model
historiesLSTM = []
# scores = []

k = 1
# for train_index, test_index in kfold.split(xtime, ytarget):
for train_index, test_index in kfold.split(xtime, ytarget):
    print("Fold: ", k)
    x_train, x_test = xtime[train_index], xtime[test_index]
    y_train, y_test = ytarget[train_index], ytarget[test_index]
    y_train = y_train.reshape(1528,14,1)
    y_test= y_test.reshape(170,14,1)
    
    historiesLSTM.append(model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=5, verbose=1))
    k += 1
#     score = model.score(x_train[test_index], y_train[test_index])
#     scores.append(score)
#     print('Fold: %s, Acc: %.3f' % (k+1, score))
    
# print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

# history = model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=50, verbose=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 90, 100)           54400     
_________________________________________________________________
lambda_2 (Lambda)            (None, 14, 100)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 14, 1)             101       
Total params: 54,501
Trainable params: 54,501
Non-trainable params: 0
_________________________________________________________________
Fold:  1
Train on 1528 samples, validate on 170 samples
Epoch 1/5
1528/1528 [==============================] - 5s 3ms/step - loss: 0.0051 - val_loss: 5.7786e-04
Epoch 2/5
1528/1528 [==============================] - 4s 2ms/step - loss: 9.2584e-04 - val_loss: 4.0997e-04
Epoch 3/5
1528/1528 [==============================] - 3s 2ms/step - loss: 6.2785e-04 - val_loss: 3.1192e-04
Epoch 4/5
1528/1528 [==

KeyboardInterrupt: 

# LSTM Validation & Training Loss

In [ ]:
from matplotlib import pyplot as plt

plt.subplot(2,2,3)
plt.plot(history.history['loss'])
plt.ylabel('MSE Training Loss')
plt.xlabel('epochs')

plt.subplot(2,2,4)
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.show()

# GRU

In [9]:
from keras.layers import LSTM, Lambda, Dropout,GRU

gru = Sequential()
gru.add(GRU(100,dropout=.2,activation='relu', input_shape=(90, 35), return_sequences=True))
gru.add(Lambda(lambda x: x[:, -predictedDays:, :]))
gru.add(Dense(1))

gru.summary()
gru.compile(optimizer='adam', loss='mse')

kfold = KFold(n_splits=10, shuffle=False, random_state=None)
# fit model
historiesGRU = []
# scores = []

k = 1
# for train_index, test_index in kfold.split(xtime, ytarget):
for train_index, test_index in kfold.split(xtime, ytarget):
    print("Fold: ", k)
    x_train, x_test = xtime[train_index], xtime[test_index]
    y_train, y_test = ytarget[train_index], ytarget[test_index]
    y_train = y_train.reshape(1528,14,1)
    y_test= y_test.reshape(170,14,1)
    
    historiesGRU.append(gru.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=5, verbose=1))
    k += 1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 90, 100)           40800     
_________________________________________________________________
lambda_3 (Lambda)            (None, 14, 100)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 14, 1)             101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
Fold:  1
Train on 1528 samples, validate on 170 samples
Epoch 1/5
1528/1528 [==============================] - 4s 2ms/step - loss: 0.0027 - val_loss: 3.5762e-04
Epoch 2/5
1528/1528 [==============================] - 3s 2ms/step - loss: 7.8702e-04 - val_loss: 3.1788e-04
Epoch 3/5
1528/1528 [==============================] - 3s 2ms/step - loss: 6.0836e-04 - val_loss: 3.1200e-04
Epoch 4/5
1528/1528 [==

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt

plt.subplot(2,2,3)
plt.plot(history.history['loss'])
plt.ylabel('MSE Training Loss')
plt.xlabel('epochs')

plt.subplot(2,2,4)
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.show()

# Stacked RNN

In [10]:

modelstacked = Sequential()
modelstacked.add(GRU(100,dropout=.2,recurrent_dropout=.2,activation='relu', input_shape=(90, 35), return_sequences=True))
modelstacked.add(GRU(100,dropout=.4,activation='relu', return_sequences=True))

modelstacked.add(Lambda(lambda x: x[:, -predictedDays:, :]))

modelstacked.add(Dense(1))

modelstacked.summary()
modelstacked.compile(optimizer='adam', loss='mse')

kfold = KFold(n_splits=10, shuffle=False, random_state=None)
# fit model
historiesStacked = []
# scores = []

k = 1
# for train_index, test_index in kfold.split(xtime, ytarget):
for train_index, test_index in kfold.split(xtime, ytarget):
    print("Fold: ", k)
    x_train, x_test = xtime[train_index], xtime[test_index]
    y_train, y_test = ytarget[train_index], ytarget[test_index]
    y_train = y_train.reshape(1528,14,1)
    y_test= y_test.reshape(170,14,1)
    
    historiesStacked.append(modelstacked.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=5, verbose=1))
    k += 1


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 90, 100)           40800     
_________________________________________________________________
gru_3 (GRU)                  (None, 90, 100)           60300     
_________________________________________________________________
lambda_4 (Lambda)            (None, 14, 100)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 14, 1)             101       
Total params: 101,201
Trainable params: 101,201
Non-trainable params: 0
_________________________________________________________________
Fold:  1
Train on 1528 samples, validate on 170 samples
Epoch 1/5
1528/1528 [==============================] - 7s 5ms/step - loss: 0.0020 - val_loss: 3.3158e-04
Epoch 2/5
1528/1528 [==============================] - 5s 3ms/step - loss: 5.4071e-04 - val_loss: 3.5705e-0

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt

plt.subplot(2,2,3)
plt.plot(historystacked.history['loss'])
plt.ylabel('MSE Training Loss')
plt.xlabel('epochs')

plt.subplot(2,2,4)
plt.plot(historystacked.history['val_loss'])
plt.xlabel('epochs')
plt.show()

# Backtracing GRU Prediction vs Real

In [ ]:
base = 1274
timeplus = 231
yhat = gru.predict(x_test[timeplus][np.newaxis,:,:])
print(yhat[0].ravel())
print(y_test[0].ravel())
print(og[base-90 + timeplus:base+14+timeplus])
# print(yhat[0] - y_test[0])
backtesting = og[base-90+timeplus:base+timeplus]
print(len(backtesting))
yhatBack=og[base-1+timeplus]
ytestBack = og[base-1+timeplus]
ytest_backtesting=og[base-90+timeplus:base+timeplus]

for i,val in enumerate(yhat[0].ravel()):
#     print(yhatBack)
    yhatBack = yhatBack * (1+val)
    
    backtesting.append(yhatBack)

for i,val in enumerate(y_test[0].ravel()):
#     print(ytestBack)
    ytestBack = ytestBack * (1+val)
    
    ytest_backtesting.append(ytestBack)
xbacktest= [x for x in range(0,104)]
plt.plot(xbacktest,backtesting,'b-')
plt.plot(xbacktest,ytest_backtesting,'r-')
plt.title("{} - {}".format(ogdates[base-90+timeplus],ogdates[base+14+timeplus]))

plt.show()


# Backtracing for LSTM and Real

In [ ]:
base = 1274
timeplus = 0
yhat = model.predict(x_test[timeplus][np.newaxis,:,:])
print(yhat[0].ravel())
print(y_test[0].ravel())
print(og[base-90 + timeplus:base+14+timeplus])
# print(yhat[0] - y_test[0])
backtesting = og[base-90+timeplus:base+timeplus]
yhatBack=og[base-1+timeplus]
ytestBack = og[base-1+timeplus]
ytest_backtesting=og[base-90+timeplus:base+timeplus]

for i,val in enumerate(yhat[0].ravel()):
#     print(yhatBack)
    yhatBack = yhatBack * (1+val)
    
    backtesting.append(yhatBack)

for i,val in enumerate(y_test[0].ravel()):
#     print(ytestBack)
    ytestBack = ytestBack * (1+val)
    
    ytest_backtesting.append(ytestBack)
xbacktest= [x for x in range(0,104)]
plt.plot(xbacktest,backtesting,'b-')
plt.plot(xbacktest,ytest_backtesting,'r-')
plt.title("{} - {}".format(ogdates[base-90+timeplus],ogdates[base+14+timeplus]))

plt.show()


# Backtracing for Stacked GRU and Real

In [ ]:
base = 1274
timeplus = 0
yhat = modelstacked.predict(x_test[timeplus][np.newaxis,:,:])
print(yhat[0].ravel())
print(y_test[0].ravel())
print(og[base-90 + timeplus:base+14+timeplus])
# print(yhat[0] - y_test[0])
backtesting = og[base-90+timeplus:base+timeplus]
yhatBack=og[base-1+timeplus]
ytestBack = og[base-1+timeplus]
ytest_backtesting=og[base-90+timeplus:base+timeplus]

for i,val in enumerate(yhat[0].ravel()):
#     print(yhatBack)
    yhatBack = yhatBack * (1+val)
    
    backtesting.append(yhatBack)

for i,val in enumerate(y_test[0].ravel()):
#     print(ytestBack)
    ytestBack = ytestBack * (1+val)
    
    ytest_backtesting.append(ytestBack)
xbacktest= [x for x in range(0,104)]
plt.plot(xbacktest,backtesting,'b-')
plt.plot(xbacktest,ytest_backtesting,'r-')
plt.title("{} - {}".format(ogdates[base-90+timeplus],ogdates[base+14+timeplus]))

plt.show()


In [ ]:
pred = model.predict(x_test[243][np.newaxis,:,:])
print(x_test[0].shape)
print(x_test.shape)

print(pred)
